# Setup

Here we import some of the library we're going to use and install the needed packages.


In [ ]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

!pip install transformers
!pip install bert-tensorflow

     |████████████████████████████████| 1.3MB 4.6MB/s 
     |████████████████████████████████| 2.9MB 30.1MB/s 
     |████████████████████████████████| 1.1MB 45.1MB/s 
     |████████████████████████████████| 890kB 47.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=8aa43552fe6d44df78f6cd5cf2bfe350d04d70b88e76139bb44cb3ef73dbda18
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 71kB 3.2MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') 
model_save_name = 'modelnamewhatever.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 

# Load Data

In [ ]:
#the file needs to have a column 'labels'
data = pd.read_excel('file.xlsx')

# Data Preparation

We will first split the the dataset in two different sets:


1.   the "training" set, which will be used to actually train the model
2.   the "validation" (and "test", here) set, which we are going to use to evaluate the model



In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset in traning and validation(test)
X_train, X_val, Y_train, Y_val = train_test_split(
    data.index.values,
    data.labels.values,
    test_size=0.06,
    random_state=17,
    stratify=data.labels.values
)

In [ ]:
# Check datasets composition
data['data_type'] = ['not_set'] * data.shape[0]
data.loc[X_train, 'data_type'] = 'train'
data.loc[X_val, 'data_type'] = 'val'
data.groupby(['labels', 'data_type']).count()

In [ ]:
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset


tokenizer = AutoTokenizer.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0", do_lower_case=True)

In [ ]:
# Encode training dataset using the tokenizer
encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type == 'train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,  # so we know when a sentence is finished
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

# Encode validation dataset using the tokenizer
encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type == 'val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,  
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

In [ ]:
# Extract IDs, attention masks and labels from training dataset
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data.data_type == 'train'].labels.values)
labels_train

In [ ]:
# Extract IDs, attention masks and labels from validation dataset
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data[data.data_type == 'val'].labels.values)

In [ ]:
# Create train and validation dataset from extracted features
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
print("Train dataset length: {}\nValidation dataset length: {}".format(len(dataset_train), len(dataset_val)))

Train dataset length: 4700
Validation dataset length: 300


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Define the size of each batch
batch_size = 32

# Load training dataset
dataloader_train= DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size)

# Load valuation dataset
dataloader_val= DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=batch_size)

In [ ]:
from transformers import BertForSequenceClassification
# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0", num_labels=2)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

# Define model optimizer -> Adam
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5, 
    eps=1e-8
)
# Define model scheduler
epochs = 8
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
import random

# Define random seeds
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# Define processor type for torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
device

In [ ]:
from sklearn.metrics import f1_score

# Returns the F1 score computed on the predictions
def f1_score_func(preds, labels):
    preds_flat=np.argmax(preds, axis=1).flatten()
    labels_flat=labels.flatten()
    return f1_score(labels_flat, preds_flat, average='binary')

# Returns the precision, accuracy and recall score computed on the predictions
def prec_func(preds, labels):
    preds_flat=np.argmax(preds, axis=1).flatten()
    labels_flat=labels.flatten()
    return precision_score(labels_flat, preds_flat)

def recall_func(preds, labels):
    preds_flat=np.argmax(preds, axis=1).flatten()
    labels_flat=labels.flatten()
    return recall_score(labels_flat, preds_flat)

def acc_func(preds, labels):
    preds_flat=np.argmax(preds, axis=1).flatten()
    labels_flat=labels.flatten()
    return accuracy_score(labels_flat, preds_flat)

In [ ]:
# Evaluates the model using the validation set
def evaluate(dataloader_val):
  model.eval()
  loss_val_total = 0
  predictions, true_vals = [], []

  for batch in dataloader_val:
      batch = tuple(b.to(device) for b in batch)
      inputs = {'input_ids': batch[0],
        'attention_mask': batch[1],
        'labels': batch[2],
        }

      with torch.no_grad():
          outputs = model(**inputs)

      loss = outputs[0]
      logits = outputs[1]
      loss_val_total += loss.item()

      logits = logits.detach().cpu().numpy()
      label_ids = inputs['labels'].cpu().numpy()
      predictions.append(logits)
      true_vals.append(label_ids)

  loss_val_avg = loss_val_total / len(dataloader_val)

  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)

  return loss_val_avg, predictions, true_vals

#Training

In [ ]:
# Training the model on the training set and checking the results on the validation
for epoch in tqdm(range(1, epochs + 1)):

    model.train()  # model is training

    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        
        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()  # to backpropagate

        torch.nn.utils.clip_grad_norm_(model.parameters(),
                                      1.0)  # prevents the gradient from being too small or too big

        optimizer.step()
        scheduler.step()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

    
    torch.save(model, path)
    tqdm.write(f'\nEpoch {epoch}/{epochs}')

    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')  # make sure that model is still training

    val_loss, predictions, true_vals = evaluate(dataloader_val)  # to check overtraining (or overfitting)
    
    val_f1 = f1_score_func(predictions, true_vals)
    val_prec = prec_func(predictions, true_vals)
    val_recall = recall_func(predictions, true_vals)
    val_acc = acc_func(predictions, true_vals)

    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score : {val_f1}')
    tqdm.write(f'Prec Score : {val_prec}')
    tqdm.write(f'Recall Score : {val_recall}')
    tqdm.write(f'Acc Score : {val_acc}')

In [ ]:
# Check the results obtained on the validation set
_, predictions, true_vals = evaluate(dataloader_val)
f1_score_func(predictions, true_vals)

#Predict

In [ ]:
#Upload file
data_test = pd.read_json('jsonfile.json', lines=True)
data_test = data_test.to_csv('namefile.csv')

In [ ]:
# Encode validation dataset using the tokenizer
encoded_data_test = tokenizer.batch_encode_plus(
    data_test.text.values,
    add_special_tokens=True,
    return_attention_mask=True,  
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

In [ ]:
# Extract IDs, attention masks and labels from validation dataset
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']

dataset_test = TensorDataset(input_ids_test, attention_masks_test)
print("Test dataset length: {}".format(len(dataset_test)))

In [ ]:
from torch.utils.data import DataLoader
dataloader_test = DataLoader(dataset_test)

In [ ]:
# Evaluates the model using the validation set
def predict(dataset_test):
    predictions = []

    for row in dataset_test:
      row = tuple(r.to(device) for r in row)
      inputs = {'input_ids': row[0],
        'attention_mask': row[1]
        }

      with torch.no_grad():
          outputs = model(**inputs)

      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      predictions.append(logits)

    return predictions

# Predict values for test dataset
predictions = predict(dataloader_test)

In [ ]:
print(len(predictions))
results = []
for i, prediction in enumerate(predictions):
  predicted = np.argmax(prediction, axis=1)[0]
  # print(f"index: {i} -- prediction: {predicted}")
  results.append(predicted)

print(results)

print(results.count(0))
print(results.count(1))

In [ ]:
misogynous = []
for prediction in results:
  misogynous.append(prediction)
data_test['misogynous'] = misogynous
data_test.to_csv('filetosave.csv')